In [1]:
import os
import pandas as pd
import numpy as np
import re
import datetime
import pathlib

# custom package(s)
from utils import utils, synapse_login

import yaml
import pathlib

with open("./local_configs/notebook_config.yaml", "r") as f:
    config = yaml.safe_load(f)

csv_model = pathlib.Path("../" + config["file_names"]["csv_model"]).resolve()
json_model = pathlib.Path("../" + config["file_names"]["json_model"]).resolve()

In [2]:
syn = synapse_login.main()


UPGRADE AVAILABLE

A more recent version of the Synapse Client (3.0.0) is available. Your version (2.7.0) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 3.0.0 release notes

https://python-docs.synapse.org/build/html/news.html



Welcome, Nicholas Lee!



In [8]:
dm_name = "../EL.data.model.csv"
output_dir = pathlib.Path("../backups").resolve()

dm_path = pathlib.Path(dm_name).resolve()
dm = utils.load_and_backup_dm(dm_path, output_dir)
dm = dm.drop(columns=["Unnamed: 0"])

In [9]:
dm.columns

Index(['Attribute', 'Description', 'Valid Values', 'Required', 'DependsOn',
       'DependsOn Component', 'Properties', 'Validation Rules', 'Template',
       'Parent', 'Source', 'Type', 'Ontology', 'multivalue', 'UsedIn',
       'module'],
      dtype='object')

In [10]:
np.sort(dm["module"].dropna().unique())

array(['analysis', 'experimentalData', 'neuro', 'ngs', 'sageCommunity'],
      dtype=object)

Updates 2023-09-06


In [ ]:
atrs_to_fix = [
    "sequencingBatchID",  # needs to accept numerical values
    "libraryVersion",  # needs to accept numerical values
]
vv = ""
vr = "regex search ^[0-9]+|(Unknown)|(Not collected)|(Not applicable)|(Not Specified)"
for a in atrs_to_fix:
    fix_index = dm_elite.query("Attribute == @a").index[0]
    dm_elite.loc[fix_index, "Valid Values"] = vv
    dm_elite.loc[dm_elite.query("Attribute == @a").index[0], "Validation Rules"] = vr

In [ ]:
dm_elite.query("Attribute in @atrs_to_fix")

In [ ]:
# Updates 9-23-2023

In [55]:
# Update descriptions
dm.loc[dm["Module"].isin(["Instrument"]), "Description"] = "Instrument Model Name"
dm.loc[dm["Module"].isin(["Study"]), "Description"] = "Project name"
dm.loc[dm["Module"].isin(["Study"]), "Module"] = "Project"

In [71]:
# Pull grant information
grants = syn.tableQuery(
    "SELECT * FROM syn51209786").asDataFrame()["grantNumber"]

grantIds = []
for i in grants:
    grantIds += i

grantIds = ",".join([i.strip() for i in grantIds])

dm.loc[
    dm["Attribute"].str.contains("grant", flags=re.IGNORECASE), "Valid Values"
] = grantIds

In [57]:
# Fix Other values
dm.loc[
    dm["Attribute"].str.contains(
        "^other|^specify", regex=True, flags=re.IGNORECASE),
    "Module",
] = "Other"

# Fix "specify" values
dm.loc[
    dm["Attribute"].str.contains(
        "^specify", regex=True, flags=re.IGNORECASE), "Parent"
] = "Specification"

In [58]:
# Fix metadata module annotation
dm.loc[
    dm["Module"].str.contains(
        "Race|Ethnicity", regex=True, flags=re.IGNORECASE, na=False
    ),
    "Module",
] = "Metadata"

In [59]:
# fix Possible values are listed under the cleavage
with pd.option_context("display.max_colwidth", None):
    display(dm[dm["Attribute"].str.contains("Possible values are")])
    display(dm[dm["Valid Values"].str.contains(
        "Possible values are", na=False)])

,Attribute,Description,Valid Values,DependsOn,Properties,Required,Parent,DependsOn Component,Source,Validation Rules,Module,Type,Ontology,UsedIn


,Attribute,Description,Valid Values,DependsOn,Properties,Required,Parent,DependsOn Component,Source,Validation Rules,Module,Type,Ontology,UsedIn


In [60]:
# fixing more values

old_values = {
    "msInstrumentModel": "Possible values are listed under the instrument model term.OtherMsInstrumentModel,Unknown,Not collected,Not applicable,Not Specified",
    "modificationParameters": "Possible values are listed under modification parameters,OtherModificationParameters,Unknown,Not collected,Not applicable,Not Specified",
    "cleavageAgents": "Possible values are listed under the cleavage agent nameOtherCleavageAgents,Unknown,Not collected,Not applicable,Not Specified	",
}

# fixing values
dm.loc[dm["Attribute"] == "msInstrumentModel", "Valid Values"] = ""
dm.loc[dm["Attribute"] == "msInstrumentModel", "Validation Rules"] = "str"

dm.loc[dm["Attribute"] == "modificationParameters", "Valid Values"] = ""
dm.loc[dm["Attribute"] == "modificationParameters", "Validation Rules"] = "str"

dm.loc[dm["Attribute"] == "cleavageAgents", "Valid Values"] = ""
dm.loc[dm["Attribute"] == "cleavageAgents", "Validation Rules"] = "str"
dm.loc[dm["Attribute"] == "cleavageAgents", "Parent"] = ""

In [61]:
# Are not actual attributes. Was a comment found in the data model
dm = dm.drop(dm[dm["Attribute"].str.contains("Possible values are")].index).reset_index(
    drop=True
)

In [62]:
# fix measurement units and change to just units
dm.loc[dm["Module"] == "Measurement Unit", "Module"] = "Unit"
dm.loc[dm["Module"] == "Unit", "Type"] = "NUMERIC"

In [65]:
# Capitallized first letter
recode_parent = {
    "dataProperty": "DataProperty",
    "specification": "Specification",
    "validValue": "ValidValue",
    "template": "Template",
}

dm["Parent"] = dm["Parent"].replace(recode_parent)

'Metadata', 'Unspecified', 'Other', 'Omics', 'Assay Metadata',
'Instrument', 'Unit', 'Assay', 'Sample Type', 'Model Organism',
'Ontology', 'Project', 'Organ', 'Boolean', 'Tissue', 'Template',
'BaseAnnotation',


In [66]:
dm.loc[
    dm["Module"].isin(["Organ", "Tissue", "Sample Type"]), "Module"
] = "Sample Metadata"

In [67]:
np.sort(dm["Module"].dropna().unique())

array(['Assay', 'BaseAnnotation', 'Instrument', 'Metadata',
       'Model Organism', 'Ontology', 'Other', 'Project',
       'Sample Metadata', 'Template', 'Unit', 'Unspecified',
       'Valid Value'], dtype=object)

In [69]:
dm.loc[dm["Module"].isin(["Omnics", "Assay Metadata"]), "Module"] = "Assay"

In [73]:
# write out data model
dm = dm[keep_cols]

dm.drop_duplicates(subset=["Attribute"], inplace=True)

dm.reset_index(drop=True, inplace=True)

In [74]:
# Updates 2023-09-21
dm.loc[dm["Attribute"] == "TRUE", "DependsOn"] = np.nan

# clean up source column
dm["Source"] = (
    dm["Source"]
    .fillna("")
    .str.split(",")
    .apply(lambda x: ",".join(sorted(np.unique(x))))
)
dm["Valid Values"] = (
    dm["Valid Values"]
    .fillna("")
    .str.split(",")
    .apply(lambda x: ",".join([y.strip() for y in x]))
)
dm["DependsOn"] = (
    dm["DependsOn"]
    .fillna("")
    .str.split(",")
    .apply(lambda x: ",".join([y.strip() for y in x]))
)

# update data model to remove not listed for purposes of this RFC
dm.loc[
    dm["Attribute"].str.contains("RFC", regex=True, flags=re.IGNORECASE, na=False),
    "Module",
] = "Ontology"

dm.loc[
    dm["Attribute"].str.contains("RFC", regex=True, flags=re.IGNORECASE, na=False),
    "Description",
] = "External ontology used for populating values"

dm.loc[
    dm["Attribute"].str.contains("RFC", regex=True, flags=re.IGNORECASE, na=False),
    "Parent",
] = "Ontology"

dm.loc[
    dm["Attribute"].str.contains("RFC", regex=True, flags=re.IGNORECASE, na=False),
    "Ontology",
] = "Self"

dm.loc[
    dm["Attribute"].str.contains("RFC", regex=True, flags=re.IGNORECASE, na=False),
    "Attribute",
] = (
    dm.loc[
        dm["Attribute"].str.contains("RFC", regex=True, flags=re.IGNORECASE, na=False),
        "Attribute",
    ]
    .str.split("(")
    .apply(lambda x: x[0].strip())
)

In [77]:
# write out new model
dm.to_csv("../EL.data.model.csv", index=False)

# convert csv model to jsonld
!schematic schema convert {csv_model} --output_jsonld {json_model}

Starting schematic...
Done adding requirements and value ranges to attributes
The Data Model was created and saved to '/Users/nlee/Documents/Projects/ELITE/ELITE-data-models/EL.data.model.jsonld' location.
